<a href="https://colab.research.google.com/github/MediciHouse07/ABM_Analysis/blob/main/Copy_of_Research_Mechanism_Design_and_GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Preface

As a practitioner and enthusiast in simulation, I have always been fascinated by modeling business, economics, politics, and more through programs. In this notebook, I will explore the potential outcomes of integrating Mechanism Design theories with Generative AI by selectively revealing information to agents and observing their reactions.

Through repeated experiments, this approach achieves three key goals:

- Gaining a fresh perspective on GenAI models.
- Understanding how AI agents respond to economic theory-driven rules.
- Most importantly—finding enjoyment in the process (and by 'you,' I really mean 'I').

This is the first step in my exploration, and I’m making it public to invite discussion. If you share a similar interest, feel free to comment and exchange ideas.
In this notebook, we will simulate a bilateral trade with two roles: buyer and seller. Each participant has a private valuation of the good. They must decide on their declared valuation. If the buyer's declared valuation is greater than the seller's, a trade occurs; otherwise, no deal is made.
The game follows the VCG (Vickrey-Clarke-Groves) mechanism, which determines the corresponding calculations, such as the transfer payments for the buyer and seller.

>Mechanism design (sometimes implementation theory or institution design)[1] is a branch of economics and game theory. It studies how to construct rules—called mechanisms or institutions—that produce good outcomes according to some predefined metric, even when the designer does not know the players' true preferences or what information they have. Mechanism design thus focuses on the study of solution concepts for a class of private-information games.

>For more information about Mechanism Design: https://en.wikipedia.org/wiki/Mechanism_design

>For more information about VCG mechanism: https://en.wikipedia.org/wiki/Vickrey%E2%80%93Clarke%E2%80%93Groves_mechanism

# II. Config rules and information to be revealed to agents

In [35]:
if_output_result = True

In [36]:
simulation_target = "VCG mechanism are revealed to both agents"
comments = ""

In [37]:
delimiter = "####"
system_message = f"""
This is a bilateral trade game following VCG rules defined in VCG mechanism. \
This is a bilateral trade game. \
In this game there are only two roles, buyer and seller. \
buyer and seller have their own evaluation for a good and that is their private information, \
in the meantime they also have declared evaluation. Both buyer and seller want to maximize their own utility. \
Remember, this is simotaneous game, buyer and seller give the declared evaluation at the same time and whether the trade would or wouldn't occur will be decided by that. \
The formulas for corresponding measurements for buyer and seller are defined as below: \
    if trade_occurs:
        price = -seller_declared_true_valuation

        # Buyer's utility
        buyer_utility = buyer_private_valuation - price

        # Seller's transfer (using declared valuations)
        seller_transfer = d * buyer_declared_valuation - buyer_declared_valuation

        # Seller's utility (using private valuation)
        seller_utility = seller_transfer + (-d * seller_private_valuation)
    else:
        price = 0
        buyer_utility = 0
        seller_utility = -d * seller_private_valuation
\
When trade occurs, d=1. \

Step 1:{delimiter} Given the information revealed, analyze what type of game it could be in terms of game theory and mechanism deisgn

Step 2:{delimiter} Think about what should be the declared evaluation given the analysis in Step 1

Step 3:{delimiter}: State your declared evaluation in the form of ##### Declared_Evaluation=Number #####
"""

Considering telling the models the game is based on VCG rules, they will simply pick the DS, this is one extreme case; remove the VCG rules from the revealed information part would take them into another extreme, from below experiment you can see that they stop telling the truth.

For the next step, we can test what would be the outcome if we only tell the agents the formulas for transfer, valuation, and utility, see if they can realize this is a VCG based game [To be added]

In [38]:
buyer_private_valuation = 5  # Buyer knows their own valuation
seller_private_valuation = 3  # Seller knows their own cost

In [39]:
buyer_prompt = f"""
You are a buyer, your private evaluation for the good is {buyer_private_valuation}""".strip()
seller_prompt = f"""
You are a seller, your private evaluation for the good is {seller_private_valuation}""".strip()

In [40]:
buyer_model = 'gemini-2.0-flash-thinking-exp'
seller_model = 'gemini-1.5-flash-8b'

# III. Technical configuration

In [41]:
# from kaggle_secrets import UserSecretsClient
from google import genai

from google.colab import userdata
secret_value_0 = userdata.get('Gemini')

# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("Gemini")

client = genai.Client(
    api_key=secret_value_0,
    http_options={'api_version': 'v1alpha'},
)

In [42]:


messages = [
    f"{system_message}",
    f"{buyer_prompt}",
]

response = client.models.generate_content(
    model=buyer_model,
    contents=messages,
)

# print(response.text)

buyer_thoughts = response.text

In [43]:


messages = [
    f"{system_message}",
    f"{seller_prompt}",
]

response = client.models.generate_content(
    model=seller_model,
    contents=messages,
)

# print(response.text)

seller_thoughts = response.text

In [44]:
import re

# Regular expression to match 'Declared_Evaluation = ' followed by a number
match = re.search(r"Declared_Evaluation\s*=\s*(\d+)", buyer_thoughts)

if match:
    buyer_declared = int(match.group(1))  # Extract the number and convert it to an integer
    print(f"Extracted number: {buyer_declared}")
else:
    print("No match found")

# Regular expression to match 'Declared_Evaluation = ' followed by a number
match = re.search(r"Declared_Evaluation\s*=\s*(\d+)", seller_thoughts)

if match:
    seller_declared = int(match.group(1))  # Extract the number and convert it to an integer
    print(f"Extracted number: {seller_declared}")
else:
    print("No match found")

Extracted number: 5
Extracted number: 3


# IV. Trade Outcome and Agents' Reasoning Steps

In [45]:
def vcg_bilateral_trade(buyer_private_valuation, seller_private_valuation,declared_valuation,declared_cost, d=1):
    """Simulates bilateral trade with the VCG mechanism.

    Args:
        buyer_private_valuation: The buyer's true valuation.
        seller_private_valuation: The seller's true cost.
        d: Externality scaling factor.

    Returns:
        Tuple: (trade_occurs, price, buyer_utility, seller_utility)
    """
    # Buyer and seller use LLMs to decide their declared valuations
    buyer_declared_valuation = declared_valuation
    seller_declared_valuation = declared_cost

    # Seller's declared valuation with externality
    seller_declared_true_valuation = -d * seller_declared_valuation

    # Trade decision based on declared valuations
    if buyer_declared_valuation >= -seller_declared_true_valuation:
        trade_occurs = True
    else:
        trade_occurs = False

    if trade_occurs:
        # VCG price based on declared valuations
        price = -seller_declared_true_valuation

        # Buyer's utility (using private valuation)
        buyer_utility = buyer_private_valuation - price

        # Seller's transfer (using declared valuations)
        seller_transfer = d * buyer_declared_valuation - buyer_declared_valuation

        # Seller's utility (using private valuation)
        seller_utility = seller_transfer + (-d * seller_private_valuation)
    else:
        price = 0
        buyer_utility = 0
        seller_utility = -d * seller_private_valuation

    return (trade_occurs, price, buyer_utility, seller_utility,
            buyer_declared_valuation, seller_declared_valuation)



results = vcg_bilateral_trade(buyer_private_valuation, seller_private_valuation, buyer_declared, seller_declared)

trade_occurs, price, buyer_utility, seller_utility, \
buyer_declared_valuation, seller_declared_valuation = results

print(f"Buyer private valuation: {buyer_private_valuation} \n")
print(f"Buyer declared valuation: {buyer_declared_valuation} \n")
print(f"Seller private valuation (inherent cost): {seller_private_valuation} \n")
print(f"Seller declared valuation (inherent cost): {seller_declared_valuation} \n")
print(f"Buyer thoughts: \n {buyer_thoughts} \n")

Buyer private valuation: 5 

Buyer declared valuation: 5 

Seller private valuation (inherent cost): 3 

Seller declared valuation (inherent cost): 3 

Buyer thoughts: 
 #### Step 1: Analyze the game
This is a bilateral trade game where trade occurs if `buyer_declared_valuation >= seller_declared_valuation`.
If trade occurs, the price is `-seller_declared_true_valuation`. Assuming there is a typo and it should be `-seller_declared_valuation`, we proceed with price as `-seller_declared_valuation`.
Buyer's utility if trade occurs is `buyer_private_valuation - price = buyer_private_valuation - (-seller_declared_valuation) = buyer_private_valuation + seller_declared_valuation`. If trade does not occur, buyer's utility is 0.
Buyer wants to maximize their utility. To have a positive utility, trade must occur. To maximize utility when trade occurs, buyer wants to maximize `seller_declared_valuation`. However, buyer cannot control seller's declared valuation directly. Buyer can only control th

In [46]:
print(f"Seller thoughts: \n {seller_thoughts} \n")

Seller thoughts: 
 Step 1: **Game Theory and Mechanism Design Analysis**

This is a simultaneous, Bayesian game with private information.  The buyer and seller each have a private valuation for the good.  The game is strategically interdependent because the decision of whether to trade depends on both the buyer and seller's declared valuations.  The VCG mechanism is a crucial component.  It is a *direct revelation* mechanism, meaning players reveal their private information directly.  It introduces an incentive structure designed to elicit truthful valuations, otherwise the mechanism itself would be vulnerable to strategic misrepresentation.

Crucially, the VCG mechanism uses a *payment rule* designed to *incentivize* truthful revelation.  The payment rule for the seller is dependent on the buyer's declared valuation.  The seller's utility is affected by both their declared and private valuations. If the seller misrepresents their valuation, their potential utility can be negatively af

In [47]:
print(f"Trade occurs: {trade_occurs} \n")
if trade_occurs:
    print(f"Price: {price}")
    print(f"Buyer utility: {buyer_utility}")
    print(f"Seller utility: {seller_utility}")
else:
    print(f"Price: {price}")
    print(f"Buyer utility: {buyer_utility}")
    print(f"Seller utility: {seller_utility}")

Trade occurs: True 

Price: 3
Buyer utility: 2
Seller utility: -3


# V. Records

In [48]:
import pandas as pd
from datetime import datetime

data = {
    'simulation_target': [simulation_target],
    'system_message': [system_message],
    'buyer_private_valuation': [buyer_private_valuation],
    'buyer_declared_valuation': [buyer_declared_valuation],
    'buyer_utility': [buyer_utility],
    'buyer_model': [buyer_model],
    'seller_private_valuation': [seller_private_valuation],
    'seller_declared_valuation': [seller_declared_valuation],
    'seller_utility':[seller_utility],
    'seller_model': [seller_model],
    'trade_occurs': [trade_occurs],
    'price': [price],
    'Timestamp': [datetime.now()],
    'comments': [comments],
    'buyer_thoughts': [buyer_thoughts],
    'seller_thoughts': [seller_thoughts]
}
df = pd.DataFrame(data)


if if_output_result:
    csv_file = 'data_log.csv'

    try:
        # Try to read the existing CSV file
        existing_df = pd.read_csv(csv_file)
        # Append the new data
        updated_df = pd.concat([existing_df, df], ignore_index=True)
    except FileNotFoundError:
        # If the file does not exist, the new data is the updated data
        updated_df = df

    # Save the updated DataFrame to the CSV file
    updated_df.to_csv(csv_file, index=False)
else:
    print("if_output_result == False")

In [49]:
import pandas as pd

# log = pd.read_csv("/kaggle/working/data_log.csv")

log = pd.read_csv("data_log.csv")

In [50]:
log

,simulation_target,system_message,buyer_private_valuation,buyer_declared_valuation,buyer_utility,buyer_model,seller_private_valuation,seller_declared_valuation,seller_utility,seller_model,trade_occurs,price,Timestamp,comments,buyer_thoughts,seller_thoughts
0,VCG mechanism are revealed to both agents,\nThis is a bilateral trade game following VCG...,5,5,2,gemini-2.0-flash-thinking-exp,3,3,-3,gemini-1.5-flash-8b,True,3,2025-02-14 05:26:59.960323,NaN,Step 1: #### Analysis of the game type\nThis i...,Step 1: **Game Type Analysis**\n\nThis is a si...
1,VCG mechanism are revealed to both agents,\nThis is a bilateral trade game following VCG...,5,5,2,gemini-2.0-flash-thinking-exp,3,3,-3,gemini-1.5-flash-8b,True,3,2025-02-14 05:28:26.703215,NaN,"Step 1:#### Given the information revealed, an...","Step 1:\n\nThis is a Bayesian game, as the buy..."
2,VCG mechanism are revealed to both agents,\nThis is a bilateral trade game following VCG...,5,5,2,gemini-2.0-flash-thinking-exp,3,3,-3,gemini-1.5-flash-8b,True,3,2025-02-14 05:29:32.613700,NaN,#### Step 1: Analyze the game\nThis is a bilat...,Step 1: **Game Theory and Mechanism Design Ana...


# X. References and useful information

https://ai.google.dev/gemini-api/docs/thinking

pip install --upgrade google-generativeai

?client.models.generate_content

With information revealed to buyer and seller as below, both of buyer and seller would delcare their true evaluation given the selected model

>This is a bilateral trade game following VCG rules defined in VCG mechanism. \
In this game there are only two roles, buyer and seller. \
buyer and seller have their own evaluation for a good and that is their private information, \
in the meantime they also have declared evaluation. Both buyer and seller want to maximize their own utility. \
Remember, this is simotaneous game, buyer and seller give the declared evaluation at the same time and whether the trade would or wouldn't occur will be decided by that

Next I will remove the VCG rules from the revealed information part